In [ ]:
!nvidia-smi

Wed May  4 08:10:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


images currently in gdrive are faulty. copy original_data.zip to disk

In [ ]:
# !ls "/content/gdrive/MyDrive/VL&R Project/data/PASCAL2012/Annotations/trainval" -1 | wc -l

Definitely want to copy images to local folder so image fetching is faster

In [ ]:
!cp "/content/gdrive/MyDrive/VL&R Project/data/PASCAL2012/original_data.zip" /content/

In [ ]:
!unzip original_data.zip

In [ ]:
!mv /content/__MACOSX/original_data/ /content/original_data

In [ ]:
!rm -rf /content/__MACOSX

In [ ]:
!ls "/content/original_data/JPEGImages/trainval" -1 | wc -l
!ls "/content/original_data/JPEGImages/test" -1 | wc -l

16227
898


# Copy augmented images to disk

In [ ]:
%cd /content

/content


In [ ]:
!ls

gdrive	original_data  original_data.zip  sample_data  vision


In [ ]:
!cp "/content/gdrive/MyDrive/VL&R Project/data/PASCAL2012/FasterRCNN-Augmented.zip" /content/

In [ ]:
!unzip FasterRCNN-Augmented.zip

# make sure ann and images have same count

In [ ]:
!ls "/content/FasterRCNN-Augmented/Annotations/trainval" -1 | wc -l

16227


In [ ]:
!ls "/content/FasterRCNN-Augmented/JPEGImages/trainval" -1 | wc -l

16227


# test an xml file

In [ ]:
# xml_path = "/content/gdrive/MyDrive/VL&R Project/data/PASCAL2012/Annotations/trainval/2007_000033.xml"

In [ ]:
xml_path = "/content/original_data/Annotations/trainval/2007_000033.xml"

In [ ]:
import os
print(os.path.exists(xml_path))

True


In [ ]:
import xml.etree.ElementTree as ET

tree = ET.parse(xml_path)
root = tree.getroot()

# size = root.findall("size")

# for s in size:
#     width = s.findall("width")[0].text
#     print(width)

b = []

for child in root:
    for t in child:
        if t.tag == 'bndbox':
            xmin = float(t.findall("xmin")[0].text)
            ymin = float(t.findall("ymin")[0].text)
            xmax = float(t.findall("xmax")[0].text)
            ymax = float(t.findall("ymax")[0].text)
            print(xmin, ymin, xmax, ymax)
            assert xmin < xmax, f"xmin not less than xmax"
            assert ymin < ymax, f"ymin not less than ymax"

9.0 107.0 499.0 263.0
421.0 200.0 482.0 226.0
325.0 188.0 411.0 223.0


# check trainval.txt to see if names are consistent with annotations or images. If not, just use names under annotations xor images folder (just a sanity check)

In [ ]:
index_path = "/content/original_data/Indexes/trainval.txt"

In [ ]:
with open(index_path, 'r') as fp:
    lines = fp.readlines()

print(len(lines))

16227


check all indices are present as images

In [ ]:
ann_dir = "/content/original_data/Annotations/trainval"
img_dir = "/content/original_data/JPEGImages/trainval"

In [ ]:
split_file = os.path.join("/content/original_data", 'Indexes', 'trainval.txt')
with open(split_file) as fp:
    index_list = [line.strip() for line in fp]

In [ ]:
for line in index_list:
    pth = os.path.join(img_dir, line + ".jpg")
    assert(os.path.exists(pth))  # should not throw any assertion errors

# Create Custom Dataset (Pascal2012)

imports

In [ ]:
import numpy as np
import os
import xml.etree.ElementTree as ET

import torch
import torch.nn
from PIL import Image
from torch.utils.data import Dataset, DataLoader

import random
import torchvision.transforms as transforms

import scipy.io
import pickle
from tqdm import tqdm

In [ ]:
import math, torch, torchvision
import torchvision.transforms.functional as F
from PIL import Image, ImageEnhance, ImageOps

def solarize_add(img, addition, threshold):
    img = F.pil_to_tensor(img)
    added_img = img + addition
    added_img = torch.clamp(added_img, 0, 255)
    return F.to_pil_image(torch.where(img < threshold, added_img, img))


def color(img, magnitude):
    return ImageEnhance.Color(img).enhance(magnitude)


def contrast(img, magnitude):
    return ImageEnhance.Contrast(img).enhance(magnitude)


def brightness(img, magnitude):
    return ImageEnhance.Brightness(img).enhance(magnitude)


def sharpness(img, magnitude):
    return ImageEnhance.Sharpness(img).enhance(magnitude)


def cutout(img, pad_size, replace):
    img = F.pil_to_tensor(img)
    _, h, w = img.shape
    center_h, center_w = torch.randint(high=h, size=(1,)), torch.randint(high=w, size=(1,))
    low_h, high_h = torch.clamp(center_h-pad_size, 0, h), torch.clamp(center_h+pad_size, 0, h)
    low_w, high_w = torch.clamp(center_w-pad_size, 0, w), torch.clamp(center_w+pad_size, 0, w)
    cutout_img = img.clone()
    cutout_img[:, low_h:high_h, low_w:high_w] = replace
    return F.to_pil_image(cutout_img)


def bbox_cutout(img, bboxs, pad_fraction, replace_with_mean):
    img = F.pil_to_tensor(img)
    _, h, w = img.shape
    random_index = torch.randint(len(bboxs), size=(1,))
    chosen_bbox = bboxs[random_index]
    min_x, min_y, max_x, max_y = chosen_bbox
    min_x, min_y, max_x, max_y = int(min_x), int(min_y), int(max_x), int(max_y)
    
    if (min_x == max_x) or (min_y == max_y): return F.to_pil_image(img)
    
    mask_x, mask_y = torch.randint(low=min_x, high=max_x, size=(1,)), torch.randint(low=min_y, high=max_y, size=(1,))
    mask_w, mask_h = pad_fraction * w / 2, pad_fraction * h / 2
    
    x_min, x_max = int(torch.clamp(mask_x-mask_w, 0, w)), int(torch.clamp(mask_x+mask_w, 0, w))
    y_min, y_max = int(torch.clamp(mask_y-mask_h, 0, h)), int(torch.clamp(mask_y+mask_h, 0, h))
    
    if replace_with_mean == True: replace = torch.mean(img[:, min_y:max_y, min_x:max_x])
    else: replace = 128
    
    cutout_img = img.clone()
    cutout_img[:, y_min:y_max, x_min:x_max] = replace
    return F.to_pil_image(cutout_img)


def _rotate_bbox(img, bboxs, degrees):
    img = F.pil_to_tensor(img)
    _, h, w = img.shape
    
    rotate_bboxs = []
    rotate_matrix = torch.FloatTensor([[math.cos(degrees*math.pi/180), math.sin(degrees*math.pi/180)], 
                                       [-math.sin(degrees*math.pi/180), math.cos(degrees*math.pi/180)]])
    for bbox in bboxs:
        min_x, min_y, max_x, max_y = bbox
        rel_min_x, rel_max_x, rel_min_y, rel_max_y = min_x-w/2, max_x-w/2, min_y-h/2, max_y-h/2
        coords = torch.FloatTensor([[rel_min_x, rel_min_y], 
                                    [rel_min_x, rel_max_y], 
                                    [rel_max_x, rel_max_y], 
                                    [rel_max_x, rel_min_y]])
        rotate_coords = torch.matmul(rotate_matrix, coords.t()).t()
        x_min, y_min = torch.min(rotate_coords, dim=0)[0]
        x_max, y_max = torch.max(rotate_coords, dim=0)[0]
        
        rotate_min_x, rotate_max_x = torch.clamp(x_min+w/2, 0, w),torch.clamp(x_max+w/2, 0, w)
        rotate_min_y, rotate_max_y = torch.clamp(y_min+h/2, 0, h),torch.clamp(y_max+h/2, 0, h)
        rotate_bboxs.append(torch.FloatTensor([rotate_min_x, rotate_min_y, rotate_max_x, rotate_max_y]))
    return torch.stack(rotate_bboxs)


def translate_bbox(img, bboxs, pixels, replace, shift_horizontal):
    img = F.pil_to_tensor(img)
    _, h, w = img.shape
    
    translate_bboxs = []
    if shift_horizontal:
        for bbox in bboxs:
            min_x, min_y, max_x, max_y = bbox
            translate_min_x, translate_max_x = torch.clamp(min_x+pixels, 0, w), torch.clamp(max_x+pixels, 0, w)
            translate_min_x, translate_max_x = int(translate_min_x), int(translate_max_x)
            translate_bboxs.append(torch.FloatTensor([translate_min_x, min_y, translate_max_x, max_y]))
    else:
        for bbox in bboxs:
            min_x, min_y, max_x, max_y = bbox
            translate_min_y, translate_max_y = torch.clamp(min_y+pixels, 0, h), torch.clamp(max_y+pixels, 0, h)
            translate_min_y, translate_max_y = int(translate_min_y), int(translate_max_y)
            translate_bboxs.append(torch.FloatTensor([min_x, translate_min_y, max_x, translate_max_y]))
    return torch.stack(translate_bboxs)


def shear_with_bboxes(img, bboxs, level, replace, shift_horizontal):
    img = F.pil_to_tensor(img)
    _, h, w = img.shape
    
    shear_bboxs = []
    if shift_horizontal:
        shear_matrix = torch.FloatTensor([[1, -level], 
                                          [0, 1]])
        for bbox in bboxs:
            min_x, min_y, max_x, max_y = bbox
            coords = torch.FloatTensor([[min_x, min_y], 
                                        [min_x, max_y], 
                                        [max_x, max_y], 
                                        [max_x, min_y]])
            shear_coords = torch.matmul(shear_matrix, coords.t()).t()
            x_min, y_min = torch.min(shear_coords, dim=0)[0]
            x_max, y_max = torch.max(shear_coords, dim=0)[0]
            shear_min_x, shear_max_x = torch.clamp(x_min, 0, w), torch.clamp(x_max, 0, w)
            shear_min_y, shear_max_y = torch.clamp(y_min, 0, h), torch.clamp(y_max, 0, h)
            shear_bboxs.append(torch.FloatTensor([shear_min_x, shear_min_y, shear_max_x, shear_max_y]))
    else:
        shear_matrix = torch.FloatTensor([[1, 0], 
                                          [-level, 1]])
        for bbox in bboxs:
            min_x, min_y, max_x, max_y = bbox
            coords = torch.FloatTensor([[min_x, min_y], 
                                        [min_x, max_y], 
                                        [max_x, max_y], 
                                        [max_x, min_y]])
            shear_coords = torch.matmul(shear_matrix, coords.t()).t()
            x_min, y_min = torch.min(shear_coords, dim=0)[0]
            x_max, y_max = torch.max(shear_coords, dim=0)[0]
            shear_min_x, shear_max_x = torch.clamp(x_min, 0, w), torch.clamp(x_max, 0, w)
            shear_min_y, shear_max_y = torch.clamp(y_min, 0, h), torch.clamp(y_max, 0, h)
            shear_bboxs.append(torch.FloatTensor([shear_min_x, shear_min_y, shear_max_x, shear_max_y]))
    return torch.stack(shear_bboxs)


def rotate_only_bboxes(img, bboxs, p, degrees, replace):
    img = F.pil_to_tensor(img)
    rotate_img = torch.zeros_like(img)

    for bbox in bboxs:
        if torch.rand(1) < p:
            min_x, min_y, max_x, max_y = bbox
            min_x, min_y, max_x, max_y = int(min_x), int(min_y), int(max_x), int(max_y)
            bbox_rotate_img = F.to_pil_image(img[:, min_y:max_y+1, min_x:max_x+1]).rotate(degrees, fillcolor=(replace,replace,replace))
            rotate_img[:, min_y:max_y+1, min_x:max_x+1] = F.pil_to_tensor(bbox_rotate_img)
    return F.to_pil_image(torch.where(rotate_img != 0, rotate_img, img))


def shear_only_bboxes(img, bboxs, p, level, replace, shift_horizontal):
    img = F.pil_to_tensor(img)
    shear_img = torch.zeros_like(img)
    
    for bbox in bboxs:
        if torch.rand(1) < p:
            min_x, min_y, max_x, max_y = bbox
            min_x, min_y, max_x, max_y = int(min_x), int(min_y), int(max_x), int(max_y)
            
            bbox_shear_img = F.to_pil_image(img[:, min_y:max_y+1, min_x:max_x+1])
            if shift_horizontal:
                bbox_shear_img = bbox_shear_img.transform(bbox_shear_img.size, Image.AFFINE, (1,level,0,0,1,0), fillcolor=(replace,replace,replace))
            else:
                bbox_shear_img = bbox_shear_img.transform(bbox_shear_img.size, Image.AFFINE, (1,0,0,level,1,0), fillcolor=(replace,replace,replace))
            shear_img[:, min_y:max_y+1, min_x:max_x+1] = F.pil_to_tensor(bbox_shear_img)

    return F.to_pil_image(torch.where(shear_img != 0, shear_img, img))


def translate_only_bboxes(img, bboxs, p, pixels, replace, shift_horizontal):
    img = F.pil_to_tensor(img)
    translate_img = torch.zeros_like(img)
    
    for bbox in bboxs:
        if torch.rand(1) < p:
            min_x, min_y, max_x, max_y = bbox
            min_x, min_y, max_x, max_y = int(min_x), int(min_y), int(max_x), int(max_y)
            
            bbox_tran_img = F.to_pil_image(img[:, min_y:max_y+1, min_x:max_x+1])
            if shift_horizontal:
                bbox_tran_img = bbox_tran_img.transform(bbox_tran_img.size, Image.AFFINE, (1,0,-pixels,0,1,0), fillcolor=(replace,replace,replace))
            else:
                bbox_tran_img = bbox_tran_img.transform(bbox_tran_img.size, Image.AFFINE, (1,0,0,0,1,-pixels), fillcolor=(replace,replace,replace))
            translate_img[:, min_y:max_y+1, min_x:max_x+1] = F.pil_to_tensor(bbox_tran_img)
    
    return F.to_pil_image(torch.where(translate_img != 0, translate_img, img))


def flip_only_bboxes(img, bboxs, p):
    img = F.pil_to_tensor(img)
    flip_img = torch.zeros_like(img)
    
    for bbox in bboxs:
        if torch.rand(1) < p:
            min_x, min_y, max_x, max_y = bbox
            min_x, min_y, max_x, max_y = int(min_x), int(min_y), int(max_x), int(max_y)
            flip_img[:, min_y:max_y+1, min_x:max_x+1] = F.hflip(img[:, min_y:max_y+1, min_x:max_x+1])
    
    return F.to_pil_image(torch.where(flip_img != 0, flip_img, img))


def solarize_only_bboxes(img, bboxs, p, threshold):
    img = F.pil_to_tensor(img)
    for bbox in bboxs:
        if torch.rand(1) < p:
            min_x, min_y, max_x, max_y = bbox
            min_x, min_y, max_x, max_y = int(min_x), int(min_y), int(max_x), int(max_y)
            solarize_img = img[:, min_y:max_y+1, min_x:max_x+1]
            solarize_img = F.to_pil_image(solarize_img)
            solarize_img = ImageOps.solarize(solarize_img, threshold=threshold)
            solarize_img = F.pil_to_tensor(solarize_img)
            img[:, min_y:max_y+1, min_x:max_x+1] = solarize_img
    return F.to_pil_image(img)


def equalize_only_bboxes(img, bboxs, p):
    img = F.pil_to_tensor(img)
    for bbox in bboxs:
        if torch.rand(1) < p:
            min_x, min_y, max_x, max_y = bbox
            min_x, min_y, max_x, max_y = int(min_x), int(min_y), int(max_x), int(max_y)
            equalize_img = img[:, min_y:max_y+1, min_x:max_x+1]
            equalize_img = F.to_pil_image(equalize_img)
            equalize_img = ImageOps.equalize(equalize_img)
            equalize_img = F.pil_to_tensor(equalize_img)
            img[:, min_y:max_y+1, min_x:max_x+1] = equalize_img
    return F.to_pil_image(img)


def cutout_only_bboxes(img, bboxs, p, pad_size, replace):
    img = F.pil_to_tensor(img)
    cutout_img = img.clone()
    
    for bbox in bboxs:
        if torch.rand(1) < p:
            min_x, min_y, max_x, max_y = bbox
            min_x, min_y, max_x, max_y = int(min_x), int(min_y), int(max_x), int(max_y)
            
            cutout_x, cutout_y = torch.randint(low=min_x, high=max_x, size=(1,)), torch.randint(low=min_y, high=max_y, size=(1,))
            
            y_min, y_max = int(torch.clamp(cutout_y-pad_size, min_y, max_y)), int(torch.clamp(cutout_y+pad_size, min_y, max_y))
            x_min, x_max = int(torch.clamp(cutout_x-pad_size, min_x, max_x)), int(torch.clamp(cutout_x+pad_size, min_x, max_x))
            
            cutout_img[:, y_min:y_max, x_min:x_max] = replace

    return F.to_pil_image(cutout_img)


import torch
import torchvision
import torchvision.transforms.functional as F

from PIL import Image, ImageOps


### Basic Augmentation
class Compose:
    """
    Composes several transforms together.
    """
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, bboxs):
        for t in self.transforms:
            image, bboxs = t(image, bboxs)
        return image, bboxs


class ToTensor:
    """
    Converts a PIL Image or numpy.ndarray (H x W x C) to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0].
    Only applied to image, not bboxes.
    """
    def __call__(self, image, bboxs):
        return F.to_tensor(image), bboxs


class Normalize(torch.nn.Module):
    """
    Normalize a tensor image with mean and standard deviation.
    Only applied to image, not bboxes.
    """
    def __init__(self, mean, std, inplace=False):
        super().__init__()
        self.mean = mean
        self.std = std
        self.inplace = inplace

    def forward(self, image, bboxs):
        return F.normalize(image, self.mean, self.std, self.inplace), bboxs


### Coior Augmentation
class AutoContrast(torch.nn.Module):
    """
    Autocontrast the pixels of the given image.
    Only applied to image, not bboxes.
    """
    def __init__(self, p):
        super().__init__()
        self.p = p

    def forward(self, image, bboxs):
      try:
        image = ImageOps.invert(image.convert('RGB'))
        autocontrast_image = ImageOps.autocontrast(image)
      except:
        autocontrast_image = image
        # if torch.rand(1) < self.p:
        #     autocontrast_image = ImageOps.autocontrast(image)
        #     return autocontrast_image, bboxs
        # else:
      return autocontrast_image, bboxs


class Brightness(torch.nn.Module):
    """
    Adjust image brightness using magnitude.
    Only applied to image, not bboxes.
    """
    def __init__(self, p, magnitude, minus=True):
        super().__init__()
        self.p = p
        self.magnitude = magnitude
        self.minus = minus

    def forward(self, image, bboxs):
        if self.minus and (torch.rand(1) < 0.5): self.magnitude *= -1
        if torch.rand(1) < self.p:
            brightness_image = brightness(image, 1+self.magnitude)
            return brightness_image, bboxs
        else:
            return image, bboxs


class Color(torch.nn.Module):
    """
    Adjust image color balance using magnitude.
    Only applied to image, not bboxes.
    """
    def __init__(self, p, magnitude, minus=True):
        super().__init__()
        self.p = p
        self.magnitude = magnitude
        self.minus = minus

    def forward(self, image, bboxs):
        if self.minus and (torch.rand(1) < 0.5): self.magnitude *= -1
        if torch.rand(1) < self.p:
            color_image = color(image, 1+self.magnitude)
            return color_image, bboxs
        else:
            return image, bboxs


class Contrast(torch.nn.Module):
    """
    Adjust image contrast using magnitude.
    Only applied to image, not bboxes.
    """
    def __init__(self, p, magnitude, minus=True):
        super().__init__()
        self.p = p
        self.magnitude = magnitude
        self.minus = minus

    def forward(self, image, bboxs):
        if self.minus and (torch.rand(1) < 0.5): self.magnitude *= -1
        if torch.rand(1) < self.p:
            contrast_image = contrast(image, 1+self.magnitude)
            return contrast_image, bboxs
        else:
            return image, bboxs


class Equalize(torch.nn.Module):
    """
    Equalize the histogram of the given image.
    Only applied to image, not bboxes.
    """
    def __init__(self, p):
        super().__init__()
        self.p = p

    def forward(self, image, bboxs):
        if torch.rand(1) < self.p:
            equalize_image = ImageOps.equalize(image)
            return equalize_image, bboxs
        else:
            return image, bboxs


class Posterize(torch.nn.Module):
    """
    Posterize the image by reducing the number of bits for each color channel.
    Only applied to image, not bboxes.
    """
    def __init__(self, p, bits):
        super().__init__()
        self.p = p
        self.bits = int(bits)

    def forward(self, image, bboxs):
        if torch.rand(1) < self.p:
            image = ImageOps.invert(image.convert('RGB'))
            posterize_image = ImageOps.posterize(image, self.bits)
            return posterize_image, bboxs
        else:
            return image, bboxs


class Sharpness(torch.nn.Module):
    """
    Adjust image sharpness using magnitude.
    Only applied to image, not bboxes.
    """
    def __init__(self, p, magnitude, minus=True):
        super().__init__()
        self.p = p
        self.magnitude = magnitude
        self.minus = minus

    def forward(self, image, bboxs):
        if self.minus and (torch.rand(1) < 0.5): self.magnitude *= -1
        if torch.rand(1) < self.p:
            sharpness_image = sharpness(image, 1+self.magnitude)
            return sharpness_image, bboxs
        else:
            return image, bboxs


class Solarize(torch.nn.Module):
    """
    Solarize the image by inverting all pixel values above a threshold.
    Only applied to image, not bboxes.
    """
    def __init__(self, p, threshold):
        super().__init__()
        self.p = p
        self.threshold = int(threshold)

    def forward(self, image, bboxs):
        if torch.rand(1) < self.p:
            solarize_image = ImageOps.solarize(image, self.threshold)
            return solarize_image, bboxs
        else:
            return image, bboxs


class SolarizeAdd(torch.nn.Module):
    """
    Solarize the image by added image below a threshold.
    Add addition amount to image and then clip the pixel value to 0~255 or 0~1.
    Parameter addition must be integer.
    Only applied to image, not bboxes.
    """
    def __init__(self, p, addition, threshold=128, minus=True):
        super().__init__()
        self.p = p
        self.addition = int(addition)
        self.threshold = int(threshold)
        self.minus = minus

    def forward(self, image, bboxs):
        if self.minus and (torch.rand(1) < 0.5): self.addition *= -1
        if torch.rand(1) < self.p:
            solarize_add_image = solarize_add(image, self.addition, self.threshold)
            return solarize_add_image, bboxs
        else:
            return image, bboxs


### Geometric Augmentation
class Rotate_BBox(torch.nn.Module):
    """
    Rotate image by degrees and change bboxes according to rotated image.
    The pixel values filled in will be of the value replace.
    Assume the coords are given min_x, min_y, max_x, max_y.
    Both applied to image and bboxes.
    """
    def __init__(self, p, degrees, replace=128, minus=True):
        super().__init__()
        self.p = p
        self.degrees = degrees
        self.replace = replace
        self.minus = minus

    def forward(self, image, bboxs):
        if self.minus and (torch.rand(1) < 0.5): self.degrees *= -1
        if torch.rand(1) < self.p:
            rotate_image = image.rotate(self.degrees, fillcolor=(self.replace, self.replace, self.replace))
            if bboxs == None:
                return rotate_image, bboxs
            else:
                rotate_bbox = _rotate_bbox(image, bboxs, self.degrees)
                return rotate_image, rotate_bbox
        else:
            return image, bboxs


class ShearX_BBox(torch.nn.Module):
    """
    Shear image and change bboxes on X-axis.
    The pixel values filled in will be of the value replace.
    Level is usually between -0.3~0.3.
    Assume the coords are given min_x, min_y, max_x, max_y.
    Both applied to image and bboxes.
    """
    def __init__(self, p, level, replace=128, minus=True):
        super().__init__()
        self.p = p
        self.level = level
        self.replace = replace
        self.minus = minus

    def forward(self, image, bboxs):
        if self.minus and (torch.rand(1) < 0.5): self.level *= -1
        if torch.rand(1) < self.p:
            shear_image = image.transform(image.size, Image.AFFINE, (1, self.level, 0, 0, 1, 0), fillcolor=(self.replace, self.replace, self.replace))
            if bboxs == None:
                return shear_image, bboxs
            else:
                shear_bbox = shear_with_bboxes(image, bboxs, self.level, self.replace, shift_horizontal=True)
                return shear_image, shear_bbox
        else:
            return image, bboxs


class ShearY_BBox(torch.nn.Module):
    """
    Shear image and change bboxes on Y-axis.
    The pixel values filled in will be of the value replace.
    Level is usually between -0.3~0.3.
    Assume the coords are given min_x, min_y, max_x, max_y.
    Both applied to image and bboxes.
    """
    def __init__(self, p, level, replace=128, minus=True):
        super().__init__()
        self.p = p
        self.level = level
        self.replace = replace
        self.minus = minus

    def forward(self, image, bboxs):
        if self.minus and (torch.rand(1) < 0.5): self.level *= -1
        if torch.rand(1) < self.p:
            shear_image = image.transform(image.size, Image.AFFINE, (1, 0, 0, self.level, 1, 0), fillcolor=(self.replace, self.replace, self.replace))
            if bboxs == None:
                return shear_image, bboxs
            else:
                shear_bbox = shear_with_bboxes(image, bboxs, self.level, self.replace, shift_horizontal=False)
                return shear_image, shear_bbox
        else:
            return image, bboxs

def translate_bbbox(img, bboxs, pixels, replace, shift_horizontal):
    img = F.pil_to_tensor(img)
    _, h, w = img.shape
    
    translate_bboxs = []
    if shift_horizontal:
        for bbox in bboxs:
            min_x, min_y, max_x, max_y = bbox
            translate_min_x, translate_max_x = np.clip(min_x+pixels, 0, w), np.clip(max_x+pixels, 0, w)
            translate_min_x, translate_max_x = int(translate_min_x), int(translate_max_x)
            translate_bboxs.append(torch.FloatTensor([translate_min_x, min_y, translate_max_x, max_y]))
    else:
        for bbox in bboxs:
            min_x, min_y, max_x, max_y = bbox
            translate_min_y, translate_max_y = np.clip(min_y+pixels, 0, h), np.clip(max_y+pixels, 0, h)
            translate_min_y, translate_max_y = int(translate_min_y), int(translate_max_y)
            translate_bboxs.append(torch.FloatTensor([min_x, translate_min_y, max_x, translate_max_y]))
    return torch.stack(translate_bboxs)

class TranslateX_BBox(torch.nn.Module):
    """
    Translate image and bboxes on X-axis.
    The pixel values filled in will be of the value replace.
    Assume the coords are given min_x, min_y, max_x, max_y.
    Both applied to image and bboxes.
    """
    def __init__(self, p, pixels, replace=128, minus=True):
        super().__init__()
        self.p = p
        self.pixels = int(pixels)
        self.replace = replace
        self.minus = minus

    def forward(self, image, bboxs):
        if self.minus and (torch.rand(1) < 0.5): self.pixels *= -1
        if torch.rand(1) < self.p:
            translate_image = image.transform(image.size, Image.AFFINE, (1, 0, -self.pixels, 0, 1, 0), fillcolor=(self.replace, self.replace, self.replace))
            if bboxs == None:
                return translate_image, bboxs
            else:
                translate_bbox = translate_bbbox(image, bboxs, self.pixels, self.replace, shift_horizontal=True)
                return translate_image, translate_bbox
        else:
            return image, bboxs


class TranslateY_BBox(torch.nn.Module):
    """
    Translate image and bboxes on Y-axis.
    The pixel values filled in will be of the value replace.
    Assume the coords are given min_x, min_y, max_x, max_y.
    Both applied to image and bboxes.
    """
    def __init__(self, p, pixels, replace=128, minus=True):
        super().__init__()
        self.p = p
        self.pixels = int(pixels)
        self.replace = replace
        self.minus = minus

    def forward(self, image, bboxs):
        if self.minus and (torch.rand(1) < 0.5): self.pixels *= -1
        if torch.rand(1) < self.p:
            translate_image = image.transform(image.size, Image.AFFINE, (1, 0, 0, 0, 1, -self.pixels), fillcolor=(self.replace, self.replace, self.replace))
            if bboxs == None:
                return translate_image, bboxs
            else:
                translate_bbox = translate_bbbox(image, bboxs, self.pixels, self.replace, shift_horizontal=False)
                return translate_image, translate_bbox
        else:
            return image, bboxs


### Mask Augmentation
class Cutout(torch.nn.Module):
    """
    Apply cutout (https://arxiv.org/abs/1708.04552) to the image.
    This operation applies a (2*pad_size, 2*pad_size) mask of zeros to a random location within image.
    The pixel values filled in will be of the value replace.
    Only applied to image, not bboxes.
    """
    def __init__(self, p, pad_size, replace=128):
        super().__init__()
        self.p = p
        self.pad_size = int(pad_size)
        self.replace = replace

    def forward(self, image, bboxs):
        if torch.rand(1) < self.p:
            cutout_image = cutout(image, self.pad_size, self.replace)
            return cutout_image, bboxs
        else:
            return image, bboxs


### Color Augmentation based on BBoxes
class Equalize_Only_BBoxes(torch.nn.Module):
    """
    Apply equalize to each bboxes in the image with probability.
    Assume the coords are given min_x, min_y, max_x, max_y.
    Only applied to image not bboxes.
    """
    def __init__(self, p):
        super().__init__()
        self.p = p/3

    def forward(self, image, bboxs):
        if bboxs == None:
            return image, bboxs
        else:
            equalize_image = equalize_only_bboxes(image, bboxs, self.p)
            return equalize_image, bboxs


class Solarize_Only_BBoxes(torch.nn.Module):
    """
    Apply solarize to each bboxes in the image with probability.
    Assume the coords are given min_x, min_y, max_x, max_y.
    Only applied to image not bboxes.
    """
    def __init__(self, p, threshold):
        super().__init__()
        self.p = p/3
        self.threshold = int(threshold)

    def forward(self, image, bboxs):
        if bboxs == None:
            return image, bboxs
        else:
            solarize_image = solarize_only_bboxes(image, bboxs, self.p, self.threshold)
            return solarize_image, bboxs


### Geometric Augmentation based on BBoxes
class Rotate_Only_BBoxes(torch.nn.Module):
    """
    Apply rotation to each bboxes in the image with probability.
    Assume the coords are given min_x, min_y, max_x, max_y.
    Only applied to image not bboxes.
    """
    def __init__(self, p, degrees, replace=128, minus=True):
        super().__init__()
        self.p = p/3
        self.degrees = degrees
        self.replace = replace
        self.minus = minus

    def forward(self, image, bboxs):
        if self.minus and (torch.rand(1) < 0.5): self.degrees *= -1
        if bboxs == None:
            return image, bboxs
        else:
            rotate_image = rotate_only_bboxes(image, bboxs, self.p, self.degrees, self.replace)
            return rotate_image, bboxs


class ShearX_Only_BBoxes(torch.nn.Module):
    """
    Apply shear to each bboxes in the image with probability only on X-axis.
    Assume the coords are given min_x, min_y, max_x, max_y.
    Only applied to image not bboxes.
    """
    def __init__(self, p, level, replace=128, minus=True):
        super().__init__()
        self.p = p/3
        self.level = level
        self.replace = replace
        self.minus = minus

    def forward(self, image, bboxs):
        if self.minus and (torch.rand(1) < 0.5): self.level *= -1
        if bboxs == None:
            return image, bboxs
        else:
            shear_image = shear_only_bboxes(image, bboxs, self.p, self.level, self.replace, shift_horizontal=True)
            return shear_image, bboxs


class ShearY_Only_BBoxes(torch.nn.Module):
    """
    Apply shear to each bboxes in the image with probability only on Y-axis.
    Assume the coords are given min_x, min_y, max_x, max_y.
    Only applied to image not bboxes.
    """
    def __init__(self, p, level, replace=128, minus=True):
        super().__init__()
        self.p = p/3
        self.level = level
        self.replace = replace
        self.minus = minus

    def forward(self, image, bboxs):
        if self.minus and (torch.rand(1) < 0.5): self.level *= -1
        if bboxs == None:
            return image, bboxs
        else:
            shear_image = shear_only_bboxes(image, bboxs, self.p, self.level, self.replace, shift_horizontal=False)
            return shear_image, bboxs


class TranslateX_Only_BBoxes(torch.nn.Module):
    """
    Apply translation to each bboxes in the image with probability only on X-axis.
    Assume the coords are given min_x, min_y, max_x, max_y.
    Only applied to image not bboxes.
    """
    def __init__(self, p, pixels, replace=128, minus=True):
        super().__init__()
        self.p = p/3
        self.pixels = int(pixels)
        self.replace = replace
        self.minus = minus

    def forward(self, image, bboxs):
        if self.minus and (torch.rand(1) < 0.5): self.pixels *= -1
        if bboxs == None:
            return image, bboxs
        else:
            translate_image = translate_only_bboxes(image, bboxs, self.p, self.pixels, self.replace, shift_horizontal=True)
            return translate_image, bboxs


class TranslateY_Only_BBoxes(torch.nn.Module):
    """
    Apply transloation to each bboxes in the image with probability only on Y-axis.
    Assume the coords are given min_x, min_y, max_x, max_y.
    Only applied to image not bboxes.
    """
    def __init__(self, p, pixels, replace=128, minus=True):
        super().__init__()
        self.p = p/3
        self.pixels = int(pixels)
        self.replace = replace
        self.minus = minus

    def forward(self, image, bboxs):
        if self.minus and (torch.rand(1) < 0.5): self.pixels *= -1
        if bboxs == None:
            return image, bboxs
        else:
            translate_image = translate_only_bboxes(image, bboxs, self.p, self.pixels, self.replace, shift_horizontal=False)
            return translate_image, bboxs


class Flip_Only_BBoxes(torch.nn.Module):
    """
    Apply horizontal flip to each bboxes in the image with probability.
    Assume the coords are given min_x, min_y, max_x, max_y.
    Only applied to image not bboxes.
    """
    def __init__(self, p):
        super().__init__()
        self.p = p/3

    def forward(self, image, bboxs):
        if bboxs == None:
            return image, bboxs
        else:
            flip_image = flip_only_bboxes(image, bboxs, self.p)
            return flip_image, bboxs


### Mask Augmentation based on BBoxes
class BBox_Cutout(torch.nn.Module):
    """
    Apply cutout to the image according to bbox information.
    Assume the coords are given min_x, min_y, max_x, max_y.
    Only applied to image, not bboxes.
    """
    def __init__(self, p, pad_fraction, replace_with_mean=False):
        super().__init__()
        self.p = p
        self.pad_fraction = pad_fraction
        self.replace_with_mean = replace_with_mean

    def forward(self, image, bboxs):
        if (torch.rand(1) < self.p) and (bboxs != None):
            cutout_image = bbox_cutout(image, bboxs, self.pad_fraction, self.replace_with_mean)
            return cutout_image, bboxs
        else:
            return image, bboxs


class Cutout_Only_BBoxes(torch.nn.Module):
    """
    Apply cutout to each bboxes in the image with probability.
    Assume the coords are given min_x, min_y, max_x, max_y.
    Only applied to image not bboxes.
    """
    def __init__(self, p, pad_size, replace=128):
        super().__init__()
        self.p = p/3
        self.pad_size = int(pad_size)
        self.replace = replace

    def forward(self, image, bboxs):
        if bboxs == None:
            return image, bboxs
        else:
            cutout_image = cutout_only_bboxes(image, bboxs, self.p, self.pad_size, self.replace)
            return cutout_image, bboxs


import torch, random

M = 10

color_range = torch.arange(0, 0.9+1e-8, (0.9-0)/M).tolist()
rotate_range = torch.arange(0, 30+1e-8, (30-0)/M).tolist()
shear_range = torch.arange(0, 0.3+1e-8, (0.3-0)/M).tolist()
translate_range = torch.arange(0, 250+1e-8, (250-0)/M).tolist()
translate_bbox_range = torch.arange(0, 120+1e-8, (120-0)/M).tolist()


Mag = {'Brightness' : color_range, 'Color' : color_range, 'Contrast' : color_range, 
       'Posterize' : torch.arange(4, 8+1e-8, (8-4)/M).tolist()[::-1], 'Sharpness' : color_range, 
       'Solarize' : torch.arange(0, 256+1e-8, (256-0)/M).tolist()[::-1], 'SolarizeAdd' : torch.arange(0, 110+1e-8, (110-0)/M).tolist(),
       
       'Cutout' : torch.arange(0, 100+1e-8, (100-0)/M).tolist(),
       
       'Rotate_BBox' : rotate_range, 'ShearX_BBox' : shear_range, 'ShearY_BBox' : shear_range,
       'TranslateX_BBox' : translate_range, 'TranslateY_BBox' : translate_range,
           
       'Rotate_Only_BBoxes' : rotate_range, 'ShearX_Only_BBoxes' : shear_range, 'ShearY_Only_BBoxes' : shear_range,
       'TranslateX_Only_BBoxes' : translate_bbox_range, 'TranslateY_Only_BBoxes' : translate_bbox_range,
       
       'Solarize_Only_BBoxes' : torch.arange(0, 256+1e-8, (256-0)/M).tolist()[::-1],
       
       'BBox_Cutout' : torch.arange(0, 0.75+1e-8, (0.75-0)/M).tolist(), 'Cutout_Only_BBoxes' : torch.arange(0, 50+1e-8, (50-0)/M).tolist()
      }


Fun = {'AutoContrast' : AutoContrast, 'Brightness' : Brightness, 'Color' : Color, 'Contrast' : Contrast, 'Equalize' : Equalize, 
       'Posterize' : Posterize, 'Sharpness' : Sharpness, 'Solarize' : Solarize, 'SolarizeAdd' : SolarizeAdd,
       
       'Cutout' : Cutout,
       
       'Rotate_BBox' : Rotate_BBox, 'ShearX_BBox' : ShearX_BBox, 'ShearY_BBox' : ShearY_BBox,
       'TranslateX_BBox' : TranslateX_BBox, 'TranslateY_BBox' : TranslateY_BBox,
           
       'Rotate_Only_BBoxes' : Rotate_Only_BBoxes, 'ShearX_Only_BBoxes' : ShearX_Only_BBoxes, 'ShearY_Only_BBoxes' : ShearY_Only_BBoxes,
       'TranslateX_Only_BBoxes' : TranslateX_Only_BBoxes, 'TranslateY_Only_BBoxes' : TranslateY_Only_BBoxes, 'Flip_Only_BBoxes' : Flip_Only_BBoxes,
       
       'Equalize_Only_BBoxes' : Equalize_Only_BBoxes, 'Solarize_Only_BBoxes' : Solarize_Only_BBoxes,
       
       'BBox_Cutout' : BBox_Cutout, 'Cutout_Only_BBoxes' : Cutout_Only_BBoxes
      }


class Policy(torch.nn.Module):
    def __init__(self, policy, pre_transform, post_transform):
        super().__init__()
        self.pre_transform = pre_transform
        self.post_transform = post_transform
        
        if policy == 'policy_v0': self.policy = policy_v0()
        elif policy == 'policy_v1': self.policy = policy_v1()
        elif policy == 'policy_v2': self.policy = policy_v2()
        elif policy == 'policy_v3': self.policy = policy_v3()
        elif policy == 'policy_vtest': self.policy = policy_vtest()

    def forward(self, image, bboxs):
        policy_idx = random.randint(0, len(self.policy)-1)
        policy_transform = self.pre_transform + self.policy[policy_idx] + self.post_transform
        policy_transform = Compose(policy_transform)
        image, bboxs = policy_transform(image, bboxs)
        return image, bboxs
    
    
def SubPolicy(f1, p1, m1, f2, p2, m2):
    subpolicy = []
    if f1 in ['AutoContrast', 'Equalize', 'Equalize_Only_BBoxes', 'Flip_Only_BBoxes']: subpolicy.append(Fun[f1](p1))
    else: subpolicy.append(Fun[f1](p1, Mag[f1][m1]))
    
    if f2 in ['AutoContrast', 'Equalize', 'Equalize_Only_BBoxes', 'Flip_Only_BBoxes']: subpolicy.append(Fun[f2](p2))
    else: subpolicy.append(Fun[f2](p2, Mag[f2][m2]))
        
    return subpolicy


def SubPolicy3(f1, p1, m1, f2, p2, m2, f3, p3, m3):
    subpolicy = []
    if f1 in ['AutoContrast', 'Equalize', 'Equalize_Only_BBoxes', 'Flip_Only_BBoxes']: subpolicy.append(Fun[f1](p1))
    else: subpolicy.append(Fun[f1](p1, Mag[f1][m1]))
    
    if f2 in ['AutoContrast', 'Equalize', 'Equalize_Only_BBoxes', 'Flip_Only_BBoxes']: subpolicy.append(Fun[f2](p2))
    else: subpolicy.append(Fun[f2](p2, Mag[f2][m2]))
        
    if f3 in ['AutoContrast', 'Equalize', 'Equalize_Only_BBoxes', 'Flip_Only_BBoxes']: subpolicy.append(Fun[f3](p3))
    else: subpolicy.append(Fun[f3](p3, Mag[f3][m3]))
        
    return subpolicy  
    

def policy_v0():
    policy = [SubPolicy('TranslateX_BBox', 0.6, 4,           'Equalize', 0.8, None),
              SubPolicy('TranslateY_Only_BBoxes', 0.2, 2,    'Cutout', 0.8, 8),
              SubPolicy('Sharpness', 0.0, 8,                 'ShearX_BBox', 0.4, 0),
              SubPolicy('ShearY_BBox', 1.0, 2,               'TranslateY_Only_BBoxes', 0.6, 6),
              SubPolicy('Rotate_BBox', 0.6, 10,              'Color', 1.0, 6)]
    return policy


def policy_v1():
    policy = [SubPolicy('TranslateX_BBox', 0.6, 4,           'Equalize', 0.8, None),
              SubPolicy('TranslateY_Only_BBoxes', 0.2, 2,    'Cutout', 0.8, 8),
              SubPolicy('Sharpness', 0, 8,                   'ShearX_BBox', 0.4, 0),
              SubPolicy('ShearY_BBox', 1.0, 2,               'TranslateY_Only_BBoxes', 0.6, 6),
              SubPolicy('Rotate_BBox', 0.6, 10,              'Color', 1.0, 6),
              SubPolicy('Color', 0.0, 0,                     'ShearX_Only_BBoxes', 0.8, 4),
              SubPolicy('ShearY_Only_BBoxes', 0.8, 2,        'Flip_Only_BBoxes', 0.0, None),
              SubPolicy('Equalize', 0.6, None,               'TranslateX_BBox', 0.2, 2),
              SubPolicy('Color', 1.0, 10,                    'TranslateY_Only_BBoxes', 0.4, 6),
              SubPolicy('Rotate_BBox', 0.8, 10,              'Contrast', 0.0, 10),
              SubPolicy('Cutout', 0.2, 2,                    'Brightness', 0.8, 10),
              SubPolicy('Color', 1.0, 6,                     'Equalize', 1.0, None),
              SubPolicy('Cutout_Only_BBoxes', 0.4, 6,        'TranslateY_Only_BBoxes', 0.8, 2),
              SubPolicy('Color', 0.2, 8,                     'Rotate_BBox', 0.8, 10),
              SubPolicy('Sharpness', 0.4, 4,                 'TranslateY_Only_BBoxes', 0.0, 4),
              SubPolicy('Sharpness', 1.0, 4,                 'SolarizeAdd', 0.4, 4),
              SubPolicy('Rotate_BBox', 1.0, 8,               'Sharpness', 0.2, 8),
              SubPolicy('ShearY_BBox', 0.6, 10,              'Equalize_Only_BBoxes', 0.6, None),
              SubPolicy('ShearX_BBox', 0.2, 6,               'TranslateY_Only_BBoxes', 0.2, 10),
              SubPolicy('SolarizeAdd', 0.6, 8,               'Brightness', 0.8, 10)]
    return policy


def policy_vtest():
    policy = [SubPolicy('TranslateX_BBox', 1.0, 4,           'Equalize', 1.0, None)]
    return policy


def policy_v2():
    policy = [SubPolicy3('Color', 0.0, 6,                    'Cutout', 0.6, 8,                 'Sharpness', 0.4, 8),
              SubPolicy3('Rotate_BBox', 0.4, 8,              'Sharpness', 0.4, 2,              'Rotate_BBox', 0.8, 10),
              SubPolicy('TranslateY_BBox', 1.0, 8,           'AutoContrast', 0.8, None),
              SubPolicy3('AutoContrast', 0.4, None,          'ShearX_BBox', 0.8, 8,            'Brightness', 0.0, 10),
              SubPolicy3('SolarizeAdd', 0.2, 6,              'Contrast', 0.0, 10,              'AutoContrast', 0.6, None),
              SubPolicy3('Cutout', 0.2, 0,                   'Solarize', 0.8, 8,               'Color', 1.0, 4),
              SubPolicy3('TranslateY_BBox', 0.0, 4,          'Equalize', 0.6, None,            'Solarize', 0.0, 10),
              SubPolicy3('TranslateY_BBox', 0.2, 2,          'ShearY_BBox', 0.8, 8,            'Rotate_BBox', 0.8, 8),
              SubPolicy3('Cutout', 0.8, 8,                   'Brightness', 0.8, 8,             'Cutout', 0.2, 2),
              SubPolicy3('Color', 0.8, 4,                    'TranslateY_BBox', 1.0, 6,        'Rotate_BBox', 0.6, 6),
              SubPolicy3('Rotate_BBox', 0.6, 10,             'BBox_Cutout', 1.0, 4,            'Cutout', 0.2, 8),
              SubPolicy3('Rotate_BBox', 0.0, 0,              'Equalize', 0.6, None,            'ShearY_BBox', 0.6, 8),
              SubPolicy3('Brightness', 0.8, 8,               'AutoContrast', 0.4, None,        'Brightness', 0.2, 2),
              SubPolicy3('TranslateY_BBox', 0.4, 8,          'Solarize', 0.4, 6,               'SolarizeAdd', 0.2, 10),
              SubPolicy3('Contrast', 1.0, 10,                'SolarizeAdd', 0.2, 8,            'Equalize', 0.2, None)]
    return policy

              
def policy_v3():
    policy = [SubPolicy('Posterize', 0.8, 2,                 'TranslateX_BBox', 1.0, 8),
              SubPolicy('BBox_Cutout', 0.2, 10,              'Sharpness', 1.0, 8),
              SubPolicy('Rotate_BBox', 0.6, 8,               'Rotate_BBox', 0.8, 10),
              SubPolicy('Equalize', 0.8, None,               'AutoContrast', 0.2, None),
              SubPolicy('SolarizeAdd', 0.2, 2,               'TranslateY_BBox', 0.2, 8),
              SubPolicy('Sharpness', 0.0, 2,                 'Color', 0.4, 8),
              SubPolicy('Equalize', 1.0, None,               'TranslateY_BBox', 1.0, 8),
              SubPolicy('Posterize', 0.6, 2,                 'Rotate_BBox', 0.0, 10),
              SubPolicy('AutoContrast', 0.6, None,           'Rotate_BBox', 1.0, 6),
              SubPolicy('Equalize', 0.0, None,               'Cutout', 0.8, 10),
              SubPolicy('Brightness', 1.0, 2,                'TranslateY_BBox', 1.0, 6),
              SubPolicy('Contrast', 0.0, 2,                  'ShearY_BBox', 0.8, 0),
              SubPolicy('AutoContrast', 0.8, None,           'Contrast', 0.2, 10),
              SubPolicy('Rotate_BBox', 1.0, 10,              'Cutout', 1.0, 10),
              SubPolicy('SolarizeAdd', 0.8, 6,               'Equalize', 0.8, None)]
    return policy

In [ ]:
# Try the original COCO mapping with pretrained weights!
coco_names = ['__background__', 'person', 'bicycle', 'car', 'motorbike', 'aeroplane', \
              'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 
              'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 
              'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella',
              'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
              'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
              'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass', 'cup', 'fork',
              'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
              'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
              'pottedplant', 'bed', 'N/A', 'diningtable', 'N/A', 'N/A', 'toilet',
              'N/A', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
              'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book', 'clock', 'vase',
              'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [ ]:
import numpy as np
import os
import xml.etree.ElementTree as ET

import torch
import torch.nn
from PIL import Image
from torch.utils.data import Dataset, DataLoader

import random
import torchvision.transforms as transforms
# from torchvision.transforms import Compose, ToTensor
import scipy.io
import pickle
from tqdm import tqdm
import albumentations as A
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import torchvision.transforms as T

class PASCALDataset(Dataset):
    CLASS_NAMES = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car',
                   'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
                   'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
    INV_CLASS = {}
    
    for i in range(len(CLASS_NAMES)):
        INV_CLASS[CLASS_NAMES[i]] = i

    CLASS_NAMES.insert(0, '__background__') # Adding background class with 0 index


    def __init__(self, split='trainval', data_dir="/content/original_data", save_lbl_pth='/content/gdrive/MyDrive/VL&R Project/data/PASCAL2012'):
        super().__init__()
        self.split      = split     # 'trainval' or 'test'
        self.data_dir   = data_dir
        self.label_pth = os.path.join(save_lbl_pth, split + "_label.pkl")
        
        self.img_dir = os.path.join(data_dir, 'JPEGImages', split)
        self.ann_dir = os.path.join(data_dir, 'Annotations', split)

        split_file = os.path.join(data_dir, 'Indexes', split + '.txt')
        with open(split_file) as fp:
            self.index_list = [line.strip() for line in fp]

        self.anno_list = self.preload_anno()

    @classmethod
    def get_class_name(cls, index):
        return cls.CLASS_NAMES[index]

    @classmethod
    def get_class_index(cls, name):
        return cls.INV_CLASS[name]

    def __len__(self):
        return len(self.index_list)

    def preload_anno(self):
        """
        :return: a list of labels. each element is in the form of [class, weight, gt_class_list, gt_boxes],
         where both class and weight are arrays/tensors in shape of [20],
         gt_class_list is a list of the class ids (separate for each instance)
         gt_boxes is a list of [xmin, ymin, xmax, ymax] values in the range 0 to 1
        """        
        if os.path.exists(self.label_pth):
            print(f"preloading from path {self.label_pth}")
            label_list = []
            with open(self.label_pth, 'rb') as fp:
                label_list = pickle.load(fp)
            
            return label_list

        label_list = []
        # print(f"reading data to save to {self.label_pth}")

        for i, index in tqdm(enumerate(self.index_list)):
            fpath = os.path.join(self.ann_dir, index + '.xml')
            tree = ET.parse(fpath)
            root = tree.getroot()

            # print(f"parsing {index}")

            C = np.zeros(20)
            W = np.ones(20) * 2 # default to enable 1 or 0 later for difficulty

            # new list for each index
            gt_class_list = []
            gt_boxes = []

            for child in root:
                
                if child.tag == 'object':
                    C[self.INV_CLASS[child[0].text]] = 1    # item at index of child name become 1
                    if child[3].text == '1' and W[self.INV_CLASS[child[0].text]] == 2:
                        W[self.INV_CLASS[child[0].text]] = 0    # if not difficult, weight is one
                    elif child[3].text == '0' :
                        W[self.INV_CLASS[child[0].text]] = 1
                    
                    # add class_index to gt_class_list
                    gt_class_list.append(self.INV_CLASS[child[0].text])

                    for t in child:
                        if t.tag == 'bndbox':
                            xmin = float(t.findall("xmin")[0].text)
                            ymin = float(t.findall("ymin")[0].text)
                            xmax = float(t.findall("xmax")[0].text)
                            ymax = float(t.findall("ymax")[0].text)
                            assert xmin < xmax, f"xmin not less than xmax for {index}"
                            assert ymin < ymax, f"ymin not less than ymax for {index}"
                            gt_boxes.append([xmin, ymin, xmax, ymax])
                    
            for i in range(len(W)):
                if W[i] == 2:
                    W[i] = 1

            label_list.append([C, W, gt_class_list, gt_boxes])
            
        # uncomment if you want to save the pickled label list to preload next time
        # with open(self.label_pth, 'wb') as fp:
        #   pickle.dump(label_list, fp)

        return label_list

    
    def __getitem__(self, index):
        """
        :param index: a int generated by Dataloader in range [0, __len__()]
        :return: index-th element - containing all the aforementioned information
        """
        # The input to the model is expected to be a list of tensors, each of shape [C, H, W], one for each image, and should be in 0-1 range.
        # Different images can have different sizes but they will be resized to a fixed size before passing it to the backbone.
        # During training, the model expects both the input tensors, as well as a targets (list of dictionary), containing:

        # boxes (FloatTensor[N, 4]): the ground-truth boxes in [x1, y1, x2, y2] format, with 0 <= x1 < x2 <= W and 0 <= y1 < y2 <= H.

        # labels (Int64Tensor[N]): the class label for each ground-truth box
        findex = self.index_list[index]     # findex refers to the file number
        fpath = os.path.join(self.img_dir, findex + '.jpg')

        # image: a PIL Image of size (H, W)
        img = Image.open(fpath).convert("RGB")
        img = transforms.ToTensor()(img)


        '''
        One note on the labels. The model considers class 0 as background.
        If your dataset does not contain the background class, you should not have 0 in your labels.
        For example, assuming you have just two classes, cat and dog, you can define 1 (not 0) to represent cats
        and 2 to represent dogs. So, for instance, if one of the images has both classes, your labels tensor should look like [1,2].

        Edit: So basically add 1 to all the label classes!
        '''

        
        # first id is the background, so remove it =====> Verify this!
        # obj_ids = obj_ids[1:]


        # target: a dict containing the following fields

        # 1. boxes (FloatTensor[N, 4]): the coordinates of the N bounding boxes in [x0, y0, x1, y1] format, ranging from 0 to W and 0 to H
        labels, boxes = self.anno_list[index][2], self.anno_list[index][3]

        aug_list = [Policy(policy='policy_v3',    pre_transform=[], post_transform=[])]

        transform = T.ToPILImage()
        img = transform(img)
        aug_image, aug_bboxes = aug_list[0](img, boxes)
        aug_image = transforms.ToTensor()(aug_image)

        boxes = torch.as_tensor(boxes, dtype=torch.float32)

        # 2. labels (Int64Tensor[N]): the label for each bounding box. 0 represents always the background class.
        labels = torch.as_tensor(labels, dtype=torch.int64) + 1 # Add 1 since background class has been added!


        # 3. image_id (Int64Tensor[1]): an image identifier.
        # It should be unique between all the images in the dataset, and is used during evaluation
        image_id = torch.tensor([index])

        # 4. area (Tensor[N]): The area of the bounding box.
        # This is used during evaluation with the COCO metric, to separate the metric scores between small, medium and large boxes.
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        # 5. iscrowd (UInt8Tensor[N]): instances with iscrowd=True will be ignored during evaluation.
        # Suppose all instances are not crowd
        iscrowd = torch.zeros((len(labels),), dtype=torch.int64)


        target = {}

        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        return aug_image, target

# Define Model (FASTER RCNN)

In [ ]:
import argparse
import os
import shutil
import time
import sys
from tkinter import N
import sklearn
import sklearn.metrics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
# model = torchvision.models.detection.ssd300_vgg16(pretrained=False, num_classes=len(PASCALDataset.CLASS_NAMES) + 1)

# load a model pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = len(PASCALDataset.CLASS_NAMES) + 1
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.train().to(device)

In [ ]:
# for name, layer in model.named_modules():
#     print(name, layer)

## some more hyperparams

In [ ]:
lr = 0.005
momentum = 0.9
weight_decay = 5e-4
epochs = 10

## define optimizer, scheduler

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1) # From the official docs

# We'll need another library for this (using this library's train function)

In [ ]:
!ls

coco_eval.py   engine.py		 presets.py   README.md  transforms.py
coco_utils.py  group_by_aspect_ratio.py  __pycache__  train.py	 utils.py


In [ ]:
!git clone https://github.com/pytorch/vision.git

In [ ]:
import os
print(os.getcwd())

/content/vision/references/detection


In [ ]:
print(os.environ['PYTHONPATH'])
os.environ['PYTHONPATH'] += ":" + "/content/vision/references/detection"
print(os.environ['PYTHONPATH'])

/env/python:/content/vision/references/detection:/content/vision/references/detection
/env/python:/content/vision/references/detection:/content/vision/references/detection:/content/vision/references/detection


In [ ]:
%cd "/content/vision/references/detection"

/content/vision/references/detection


# Train Function

In [ ]:
# Import functions from inside the /content/vision/references/detection path
from engine import train_one_epoch, evaluate
import utils

# Define dataset and dataloader

In [ ]:
# We require utils for the collate function
torch.cuda.empty_cache()

batch_size = 4
num_workers = 4


train_dataset = PASCALDataset('trainval', data_dir="/content/original_data")
train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True,num_workers=num_workers,pin_memory=True,collate_fn=utils.collate_fn)
print('\n', len(train_loader))

16227it [00:01, 9879.05it/s]


 4057



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
test_dataset = PASCALDataset('test', data_dir="/content/original_data")
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False,num_workers=num_workers,pin_memory=True,collate_fn=utils.collate_fn)
print('\n', len(test_loader))

898it [00:00, 9148.33it/s]


 225



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# Runner

In [ ]:
for epoch in range(epochs):
    # train for one epoch

    train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=100)

    # VIVEK ADD EVALUATION Done! \(^_^)/
    evaluate(model, test_loader, device=device)

    scheduler.step()

# save_path = os.path.join("/content/gdrive/MyDrive/VL&R Project/pascal-training/saved_models", f"faster-rcnn_epochs{epochs}.pt")
# torch.save(model, save_path)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: [0]  [   0/4057]  eta: 1:27:07  lr: 0.000010  loss: 3.9357 (3.9357)  loss_classifier: 3.5577 (3.5577)  loss_box_reg: 0.3220 (0.3220)  loss_objectness: 0.0087 (0.0087)  loss_rpn_box_reg: 0.0472 (0.0472)  time: 1.2884  data: 0.4637  max mem: 10941
Epoch: [0]  [ 100/4057]  eta: 0:44:50  lr: 0.000509  loss: 0.5594 (1.2817)  loss_classifier: 0.2877 (0.9726)  loss_box_reg: 0.2436 (0.2793)  loss_objectness: 0.0082 (0.0147)  loss_rpn_box_reg: 0.0110 (0.0152)  time: 0.6718  data: 0.0002  max mem: 10941
Epoch: [0]  [ 200/4057]  eta: 0:43:22  lr: 0.001009  loss: 0.4504 (0.9392)  loss_classifier: 0.2077 (0.6299)  loss_box_reg: 0.2419 (0.2790)  loss_objectness: 0.0055 (0.0147)  loss_rpn_box_reg: 0.0101 (0.0156)  time: 0.6744  data: 0.0002  max mem: 10941
Epoch: [0]  [ 300/4057]  eta: 0:41:54  lr: 0.001508  loss: 0.3714 (0.7775)  loss_classifier: 0.1835 (0.4896)  loss_box_reg: 0.1798 (0.2578)  loss_objectness: 0.0103 (0.0147)  loss_rpn_box_reg: 0.0098 (0.0153)  time: 0.6382  data: 0.0008  max

In [ ]:
# save state dict (just in case)
# save_path = os.path.join("/content/gdrive/MyDrive/VL&R Project/pascal-training/saved_models", f"faster-rcnn-statedict_epochs{epochs}.pt")
# torch.save(model.state_dict(), save_path)

In [ ]:
# to load model again
model = torch.load(save_path)
model.eval()